**Note: This is my EDA.  It's not pretty or nice and probably has some mistakes along the way.  Check out the associated `.py` file for a linted/formatted version of the solution.**

In [40]:
import datetime
from pathlib import Path
import sys
import math

import polars as pl

# Gross hack to get jupyter to find hod stuff.
sys.path.append(str(Path(Path.cwd().parent)))
from hod.utils import get_data, get_male_names

pl.Config.set_tbl_rows(n=100)

polars.config.Config

# The Meet Cute

(JAMES: Emphasis below mine when I think something will be useful.)

REF: https://hanukkah.bluebird.sh/5784/7/


“Oh that tapestry, with the colorful toucan on it! I’ll tell you what happened to it.

“One day, I was at Noah’s Market, and I was just about to leave when someone behind me said ‘Miss! You dropped something!’

“Well I turned around to see this cute guy holding an item I had bought. He said, ‘I got the same thing!’ We laughed about it and wound up swapping items because I wanted the color he got. We had a moment when our eyes met and my heart stopped for a second. I asked him to get some food with me and we spent the rest of the day together.

“Before long I moved into his place, but the romance faded quickly, as he wasn’t the prince I imagined. I left abruptly one night, forgetting the tapestry on his wall. But by then, it symbolized our love, and I wanted nothing more to do with it. For all I know, he still has it.”

Can you figure out her ex-boyfriend’s phone number?

In [9]:
data = get_data(data_path=Path('./../data/5784/'))

df_customers = data["customers"]
df_orders = data["orders"]
df_orders_items = data["orders_items"]
df_products = data["products"]

colors = ["green","mauve","blue","orange","yellow","purple","amber","puce","magenta","azure","white","red"]
color_str_regex = "|".join(colors)

df_color_products = df_products.filter((
    ~pl.col("sku").str.starts_with("PET"),
    ~pl.col("sku").str.starts_with("DLI"),
    (
        pl.col("desc").str.contains(color_str_regex)
    )
))

df_orders_of_color_products = df_color_products.join(df_orders_items, on="sku").join(df_orders, on="orderid").drop(["dims_cm", "wholesale_cost", "qty", "unit_price", "shipped", "items", "total"]).with_columns(pl.col("ordered").dt.date().alias("date")).with_columns(pl.col("ordered").dt.hour().alias("hour")).with_columns(pl.col("desc").str.extract("\((.*?)\)").alias("color"))

df_suspects = df_orders_of_color_products.join(df_customers, on="customerid").drop(["address", "citystatezip", "birthdate", "phone", "timezone", "lat", "long"])

# Dates where Sherri Long.
sus_dates = (
    df_suspects
    .group_by(["date"])
    .agg([pl.col("name"), "color"])
    .filter(pl.col("name").list.contains("Sherri Long"))
    .get_column("date")
)

# Find someone closest to Sherri Long in time.
df_suspects.filter(pl.col("date").is_in(sus_dates)).select(["customerid", "name", "desc", "ordered", "color"]).sort("ordered")

df_customers.filter(pl.col("customerid") == 5783)

